In [2]:
import torch
import evaluate
import numpy as np
from datasets import load_from_disk
from transformers import Trainer, TrainingArguments, ViTForImageClassification
from sklearn.model_selection import KFold

# Load Dataset

In [3]:
dataset = load_from_disk("../data/processed/huggingface")
labels = dataset["train"].features["label"].names

In [21]:
dataset["train"]

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 1814
})

In [22]:
dataset["test"]

Dataset({
    features: ['image', 'label', 'pixel_values'],
    num_rows: 454
})

# Training

k-fold cross-validation with incremental learning

In [4]:
accuracy = evaluate.load("accuracy")

def compute_accuracy(p):
    pred = np.argmax(p.predictions, axis=1)
    lab = p.label_ids
    return accuracy.compute(predictions=pred, references=lab)

results_folder = "../models/results"
logs_folder = "../models/logs"

# Define training arguments
training_args = TrainingArguments(
    output_dir= results_folder,
    evaluation_strategy="epoch",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_strategy="no",
    logging_dir=logs_folder,
    logging_steps=10,
)

# Initialize with pre-trained model
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224",
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)},
    ignore_mismatched_sizes=True,  # Ignore the classifier's size mismatch
)

# Initialize k-fold splits
k = 5
kf = KFold(n_splits=k, shuffle=True)
all_fold_metrics = []

# Perform k-fold training
for fold, (train_idx, val_idx) in enumerate(kf.split(dataset["train"])):
    print(f"Training fold {fold+1}/{k}")

    train_split = dataset["train"].select(train_idx)
    val_split = dataset["train"].select(val_idx)

    training_args.output_dir = f"{results_folder}/fold_{fold}"
    training_args.logging_dir = f"{logs_folder}/fold_{fold}"

    # Trainer instance for this fold
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_split,
        eval_dataset=val_split,
        compute_metrics=compute_accuracy
    )

    trainer.train()
    eval_result = trainer.evaluate()

    print(eval_result)
    all_fold_metrics.append(eval_result)

model.save_pretrained(f"{results_folder}/final_model")

# Evaluate final model on test set
print("Evaluating final model on test set")
test_result = trainer.evaluate(eval_dataset=dataset["test"])
print(test_result)

/home/calvin/Projects/icon-classifier/.venv/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([63]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([63, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Training fold 1/5


  5%|▌         | 10/182 [04:25<1:16:50, 26.80s/it]

{'loss': 4.1508, 'grad_norm': 7.471713542938232, 'learning_rate': 4.7252747252747257e-05, 'epoch': 0.11}


 11%|█         | 20/182 [08:53<1:13:33, 27.24s/it]

...

{'eval_loss': 7.254808588186279e-05, 'eval_accuracy': 1.0, 'eval_runtime': 164.4935, 'eval_samples_per_second': 2.201, 'eval_steps_per_second': 0.14, 'epoch': 2.0}
{'train_runtime': 4080.9169, 'train_samples_per_second': 0.712, 'train_steps_per_second': 0.045, 'train_loss': 0.00015888688686063287, 'epoch': 2.0}


100%|██████████| 23/23 [02:35<00:00,  6.78s/it]


{'eval_loss': 7.254808588186279e-05, 'eval_accuracy': 1.0, 'eval_runtime': 164.7694, 'eval_samples_per_second': 2.197, 'eval_steps_per_second': 0.14, 'epoch': 2.0}
Evaluating final model on test set


100%|██████████| 29/29 [03:17<00:00,  6.82s/it]

{'eval_loss': 0.012209120206534863, 'eval_accuracy': 0.9977973568281938, 'eval_runtime': 206.7976, 'eval_samples_per_second': 2.195, 'eval_steps_per_second': 0.14, 'epoch': 2.0}


# Inference

Test with a single example from "test" dataset

In [20]:
# Pick and example
ex = dataset["test"][42]
ex_px_data = torch.tensor(ex["pixel_values"]).unsqueeze(0)  # Add batch dimension
expected_class_label = model.config.id2label[str(ex["label"])]

# Perform inference
with torch.no_grad():  # Disable gradient computation during inference
    outputs = model(ex_px_data)
    logits = outputs.logits

# Convert logits to probabilities and get predicted class
probs = torch.softmax(logits, dim=-1)
predicted_class_idx = logits.argmax(-1).item()
predicted_class_label = model.config.id2label[str(predicted_class_idx)]

print(f"Predicted label '{expected_class_label}' is correct: [{expected_class_label == predicted_class_label}]")

Predicted label '2047' is correct: [True]


In [27]:
probs[:,predicted_class_idx].item()

0.9999359846115112